In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Debug"
sys.path.append(atlas_path)


from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *
from AtlasPy.model import *

In [2]:
exchange_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/src/exchangeVBT"
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [3]:
st = time.perf_counter_ns()
n = 2
close = AssetReadNode.make("Close", 0, exchange)
open = AssetReadNode.make("Open", 0, exchange)
prev_close = AssetReadNode.make("Close", -1, exchange)

feat1 = AssetOpNode.make(
    close,
    prev_close,
    AssetOpType.SUBTRACT
)
feat2 = AssetOpNode.make(
    close,
    open,
    AssetOpType.SUBTRACT
)
target = ModelTarget(
    close,
    ModelTargetType.ABSOLUTE,
    2
)

In [4]:
config = ModelConfig(
    training_window = 5,
    walk_forward_window = 3,
    model_type = ModelType.LINEAR_REGRESSION,
    exchange = exchange,
)
lr_config = LinearRegressionModelConfig(
    config,
    LinearRegressionSolver.LDLT,
    True
)
lr_model = LinearRegressionModel(
    "lr_model",
    [feat1, feat2],
    target,
    lr_config
)
exchange.registerModel(lr_model)

for i in range(4):
    hydra.step()

In [5]:
lr_model.getX()

array([[1.32489941e+03, 1.35709961e+03, 1.00000000e+00],
       [2.18900391e+02, 2.22799805e+02, 1.00000000e+00],
       [3.98200195e+02, 3.28500000e+02, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [6]:
lr_model.getY()

array([15599.20019531,     0.        ,     0.        ,     0.        ,
           0.        ])

In [7]:
ticker = "BTC-USD"
asset_id = exchange.getAssetIndex(ticker)
path = os.path.join(exchange_path,f"{ticker}.csv")  
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df["Date"]) 
df = df.set_index("Date")
df = df[['Open', 'High', 'Low', 'Close']]
df["feat_1"] = df["Close"] - df["Close"].shift(1)
df["feat_2"] = df["Close"] - df["Open"]
df

,Open,High,Low,Close,feat_1,feat_2
Date,,,,,,
2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,NaN,-455.000000
2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,1324.899414,1357.099609
2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,218.900391,222.799805
2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,398.200195,328.500000
2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,1830.299805,1952.299805
...,...,...,...,...,...,...
2023-12-28 00:00:00+00:00,43468.199219,43804.781250,42318.550781,42627.855469,-815.000000,-840.343750
2023-12-29 00:00:00+00:00,42614.644531,43124.324219,41424.062500,42099.402344,-528.453125,-515.242188
2023-12-30 00:00:00+00:00,42091.753906,42584.125000,41556.226562,42156.902344,57.500000,65.148438


In [8]:
"""
ticker = "BTC-USD"
asset_id = exchange.getAssetIndex(ticker)
path = os.path.join(exchange_path,f"{ticker}.csv")  
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df["Date"]) 
df = df.set_index("Date")

def ts_argmax(df, window=10):
    return df.rolling(window).apply(np.argmax).add(1)

btc_idx = exchange.getAssetIndex("BTC-USD")
df["close_arg_max_atlas"] = close_arg_max.cache()[btc_idx].T
df["close_max_atlas"] = close_max.cache()[btc_idx].T
df["close_max_pd"] = df["Close"].diff().rolling(n).max()
df["close_arg_max_pd"] = ts_argmax(df["Close"].diff(), n)
df["change"] = df["Close"].diff()
df = df[["Close","change", "close_max_atlas", "close_max_pd", "close_arg_max_pd", "close_arg_max_atlas"]]
df.head(30)
"""

'\nticker = "BTC-USD"\nasset_id = exchange.getAssetIndex(ticker)\npath = os.path.join(exchange_path,f"{ticker}.csv")  \ndf = pd.read_csv(path)\ndf["Date"] = pd.to_datetime(df["Date"]) \ndf = df.set_index("Date")\n\ndef ts_argmax(df, window=10):\n    return df.rolling(window).apply(np.argmax).add(1)\n\nbtc_idx = exchange.getAssetIndex("BTC-USD")\ndf["close_arg_max_atlas"] = close_arg_max.cache()[btc_idx].T\ndf["close_max_atlas"] = close_max.cache()[btc_idx].T\ndf["close_max_pd"] = df["Close"].diff().rolling(n).max()\ndf["close_arg_max_pd"] = ts_argmax(df["Close"].diff(), n)\ndf["change"] = df["Close"].diff()\ndf = df[["Close","change", "close_max_atlas", "close_max_pd", "close_arg_max_pd", "close_arg_max_atlas"]]\ndf.head(30)\n'